## Семинар 5: Proximal Policy Optimization 

В практической реализации существует два варианта реализации алгоритма PPO:
* выполняет обновление, ограниченное KL, как TRPO, но штрафует KL-расхождение в целевой функции вместо того, чтобы делать его жестким ограничением, и автоматически регулирует коэффициент штрафа в процессе обучения, чтобы он масштабировался соответствующим образом.
* не содержит в целевой функции члена KL-дивергенции и вообще не имеет ограничения. Вместо этого полагается на специализированный клиппинг 

<img src="https://spinningup.openai.com/en/latest/_images/math/e62a8971472597f4b014c2da064f636ffe365ba3.svg">

Спойлер: клиппинг - не самое главное в PPO, как это могло показаться на первый взгляд. Алгоритм PPO работает во многом и за счет небольших дополнительных улучшений. Подробнее: https://arxiv.org/pdf/2005.12729.pdf

In [ ]:
!pip install gym[Box2D] --force

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gym
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

### Сеть Actor-Critic

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, action_dim),
            nn.Softmax(dim=-1)
        )

        # critic
        self.value_layer = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, 1)
        )

    def forward(self):
        raise NotImplementedError

    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device)
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()

        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))

        return action.item()

    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()

        state_value = self.value_layer(state)
        return action_logprobs, torch.squeeze(state_value), dist_entropy


### PPO policy

In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()

    def update(self, memory):
        # Monte Carlo оценка вознаграждений:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)

        # нормализация вознаграждений:
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)

        # конвертация list в tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()

        # оптимизация K epochs:
        for _ in range(self.K_epochs):
            # оцениваем старую стратегию:
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # находим отношение стратегий (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Находим Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5 * self.MseLoss(state_values, rewards) - 0.01 * dist_entropy
            # делаем шаг градиента
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # копируем веса
        self.policy_old.load_state_dict(self.policy.state_dict())

### Гиперпараметры

In [ ]:
env_name = "LunarLander-v2"

env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
render = False
solved_reward = 200  # останавливаемся если avg_reward > solved_reward
log_interval = 20  # печатаем avg reward  в интервале 
max_episodes = 50000  # количество эпизодов обучения
max_timesteps = 500  # максимальное кол-во шагов в эпизоде
n_latent_var = 64  # кол-во переменных в скрытых слоях
update_timestep = 2000  # обновляем policy каждые n шагов
lr = 0.001 # learning rate
betas = (0.9, 0.999) # betas для adam optimizer
gamma = 0.99  # discount factor
K_epochs = 4  # количество эпох обноеления policy
eps_clip = 0.1  # clip параметр для PPO
random_seed = None

### Основной цикл

In [ ]:
if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
print(lr, betas)

# переменные для логирования
running_reward = 0
avg_length = 0
timestep = 0

# цикл обучения
for i_episode in range(1, max_episodes + 1):
    state = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # запускаем policy_old:
        action = ppo.policy_old.act(state, memory)
        state, reward, done, _ = env.step(action)

        # сохраняем reward and is_terminal:
        memory.rewards.append(reward)
        memory.is_terminals.append(done)

        # делаем обновление
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if render:
            env.render()
        if done:
            break

    avg_length += t

    # останавливаем обучение если avg_reward > solved_reward
    if running_reward > (log_interval * solved_reward):
        print("########## Принято! ##########")
        torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        break

    # логирование
    if i_episode % log_interval == 0:
        avg_length = int(avg_length / log_interval)
        running_reward = int((running_reward / log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
        running_reward = 0
        avg_length = 0

### Задание 1: В данном семинаре вашей задачей будет среда LunarLander-v2, она сложнее базовых сред, поэтому вам будет необходимо изменить некотороые гиперпараметры.

### Задание 2: Сравните результаты с начальными гиперпараметрами. Как вы считате, что повлияло больше всего?